# Análise comparativa

O objetivo dessa eanalise é comparar modelos para encontrar um que possa ser melhor utilizado dentro do problema em questão (estimar a idade por meio do numero de aneis), para isso antes realizamos a preparação e o pré-processamento dos dados.

## Preparação dos dados

### 1.1 Configurações iniciais

Importações e configurações

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_predict, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, train_test_split

from sklearn.model_selection import (
    cross_validate, GridSearchCV, ShuffleSplit
)

### 1.2 Obtendo dados

Nessa etapa vamos importar arquivos brutos de dados e o dicionário antes de começarmos o pre-processamento

In [2]:
#Importando dados
names = ['Sexo', 'Comprimento', 'Diâmetro', 'Altura', 'Peso total', 'Peso sem concha', 'Peso do intestino', 'Peso da concha', 'Anéis']
df = pd.read_csv("https://raw.githubusercontent.com/atlantico-academy/equipe-01/master/data/raw/abalone.csv", header=None, names=names)

In [3]:
df.head()

,Sexo,Comprimento,Diâmetro,Altura,Peso total,Peso sem concha,Peso do intestino,Peso da concha,Anéis
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Obtendo informações sobre o conjunto de dados

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Sexo               4177 non-null   object 
 1   Comprimento        4177 non-null   float64
 2   Diâmetro           4177 non-null   float64
 3   Altura             4177 non-null   float64
 4   Peso total         4177 non-null   float64
 5   Peso sem concha    4177 non-null   float64
 6   Peso do intestino  4177 non-null   float64
 7   Peso da concha     4177 non-null   float64
 8   Anéis              4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [5]:
df.isna().sum()

Sexo                 0
Comprimento          0
Diâmetro             0
Altura               0
Peso total           0
Peso sem concha      0
Peso do intestino    0
Peso da concha       0
Anéis                0
dtype: int64

O conjunto de dados é composto de 8 variáveis de entrada (7 contínuas e 1 categórica) e 1 variável de saída (inteira). Ele não possui dados faltantes.

Definindo os parametros do dataset de entrada e saida

In [6]:
nominal_columns = ['Sexo']
continuos_columns = ['Comprimento', 'Diâmetro', 'Altura', 'Peso total', 'Peso sem concha', 'Peso do intestino', 'Peso da concha']
target_column = ['Anéis']

In [7]:
# Fizemos aqui a um drop (exclusão temporaria) da coluna aneis para iniciar o modelo entrada (oque a maquina vai interpretar)
# e saida( oque ela vai tentar adivinhar)
X = df.drop(df[target_column], axis=1)
y = df[target_column]

In [8]:
X.head()

,Sexo,Comprimento,Diâmetro,Altura,Peso total,Peso sem concha,Peso do intestino,Peso da concha
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055


In [9]:
y.head()

,Anéis
0,15
1,7
2,9
3,10
4,7


### 1.3 Tratamento de dados

Aqui realizamos a normalização e codificação de variáveis categóricas

#### 1.3.1 - Tratamento de dados discrepantes

Não realizamos, por ter feito testes e nenhum dado ter sido removido 

### 1.4 Codificação das variáveis categóricas e Normalização dos dados.

Iremos utilizar o metodo de normalização de dados chamado:

- Z-Score

#### Criando pipeline

In [10]:
nominal_preprocessor = Pipeline(steps=[
    # Codificação das variáveis
    ('encoding', OneHotEncoder())
])
continuous_preprocessor = Pipeline(steps=[
    # Normalização
    ('normalization', StandardScaler(with_mean=False))
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuos', continuous_preprocessor, continuos_columns)
])

## Escolha do modelo

### 2.1 Metodologia

Iremos análisar quatro modelos, que serão testados utilizando um método de validação cruzada por permutação, os modelos que serão testados serão:

- Regressão linear simples (OLS)
- Regressão Penalizada Ridge (RR)
- K-Nearest-Neighbors (KNN)
- Support Vector Regressor (SVR)

Além disso, cada um desses algoritmos será testado com diferentes parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

### 2.2 Modelos Lineares de Regressão

- Regressão linear simples (OLS)
- Regressão Penalizada Ridge (RR)

Para o modelo de Regressão linear simples (OLS):
- hiperparametro vazio () por não haver necessidade, pois por padrão ele usa o metodo de Mínimos Quadrados Ordinários)

Para o modelo de Regressão Penalizada Ridge (RR):
- hiperparametro 'alpha' adotado como np.arange(0.01, 1.0, 0.01) especifica uma sequência de valores de 0.01 a 1.0 (exclusivo) em incrementos de 0.01 para serem testados durante a busca em grade.

In [53]:
linear_models = [
    (
        "OLS",
        LinearRegression(),
        {
            
        }

    ),
    (
        "RR",
        Ridge(),
        {
            "alpha": np.arange(0.01, 5, 0.01)
        }
    )
]

Após alguns testes prévios trocando os valores dos parametros, chegamos na seguinte otimizazção:
- Nº de repetições 21 ( foram testados valores impares de 1 a 21, sendo 21 o de maior resultado em ambas)
- adotamos o coeficiente de determinação r2 para avaliar o desempenho
- Utilizamos validação cruzada por permutação com 21 divisões e um tamanho de teste de 20%

In [54]:
n_splits = 21
final_results_lm = {}
best_params_lm = {}

for model_name, model, model_hparams_grid in linear_models:
    print(f"{model_name} is running...")
    model_gs = GridSearchCV(
        model,
        model_hparams_grid,
        scoring='neg_root_mean_squared_error',
        cv=5
    )
    approach = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model_gs)
    ])
    results = cross_validate(
        approach,
        X=X,
        y = y.to_numpy().ravel(),
        scoring=[
            "neg_root_mean_squared_error",
            "r2"
        ],
        cv=ShuffleSplit(n_splits=n_splits, test_size=.2)
    )

    approach.fit(X, y.to_numpy().ravel())
    
    best_params = model_gs.best_params_
    best_params_lm[model_name] = best_params


    results["name"] = [model_name] * n_splits
    results["test_root_mean_squared_error"] = np.abs(results.pop("test_neg_root_mean_squared_error"))
    if final_results_lm:
        for key, value in results.items():
            final_results_lm[key] = np.append(final_results_lm[key], value)
    else:
        final_results_lm = results
    
for model_name, best_params in best_params_lm.items():
    print(f"Best Parameters for {model_name}: {best_params}")

OLS is running...
RR is running...
Best Parameters for OLS: {}
Best Parameters for RR: {'alpha': 4.99}


### 2.3 Modelos Não-Lineares de Regressão

- K-Nearest-Neighbors (KNN)
- Support Vector Regressor (SVR)

Para o Modelo K-Nearest Neighbors (KNN):
- intervalo de numeros impares de 1 a 50
- modelo de pesos é uniforme e que vizinhos que estejam mais proximos tem maior valor

Para o Modelo Support Vector Regressor (SVR):
- Analisando os dados optamos por escolher a seguinte configuração do kernel:
  - Radial Basis Function (RBF) - "rbf"
  - Polynomial - "poly"
  - Sigmoid - "sigmoid"

In [49]:
nonlinear_models = [
    (
        "KNN",
        KNeighborsRegressor(),
        {
            "n_neighbors": np.arange(1, 50, 2),
            "weights": ["uniform","distance"]
        }
    ),
    
    (
        "SVR",
        SVR(max_iter=100000),
        {
            "kernel": ["rbf", "poly", "sigmoid"]
        }
    )
]

Após alguns testes prévios trocando os valores dos parametros, chegamos na seguinte otimizazção:
- Nº de repetições 21 ( foram testados valores de 1 a 21, sendo 21 o de maior resultado em ambas)
- adotamos o coeficiente de determinação r2 para avaliar o desempenho
- Utilizamos validação cruzada por permutação com 21 divisões e um tamanho de teste de 20%

In [50]:
n_splits = 21
final_results_nlm = {}
for model_name, model, model_hparams_grid in nonlinear_models:
    print(f"{model_name} is running...")
    model_gs = GridSearchCV(
        model,
        model_hparams_grid,
        scoring='neg_root_mean_squared_error',
        cv=5
    )
    approach = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model_gs)
    ])
    results = cross_validate(
        approach,
        X=X,
        y = y.to_numpy().ravel(),
        scoring=[
            "neg_root_mean_squared_error",
            "r2"
        ],
        cv=ShuffleSplit(n_splits=n_splits, test_size=.2)
    )

    approach.fit(X, y.to_numpy().ravel())
    
    best_params = model_gs.best_params_
    best_params_lm[model_name] = best_params
    
    results["name"] = [model_name] * n_splits
    results["test_root_mean_squared_error"] = np.abs(results.pop("test_neg_root_mean_squared_error"))
    if final_results_nlm:
        for key, value in results.items():
            final_results_nlm[key] = np.append(final_results_nlm[key], value)
    else:
        final_results_nlm = results

for model_name, best_params in best_params_lm.items():
    print(f"Best Parameters for {model_name}: {best_params}")

KNN is running...
SVR is running...
Best Parameters for OLS: {}
Best Parameters for RR: {'alpha': 4.99}
Best Parameters for KNN: {'n_neighbors': 15, 'weights': 'distance'}
Best Parameters for SVR: {'kernel': 'rbf'}


### 2.4 Resultados da experimentação

#### 2.4.1 Modelos Lineares

In [55]:
(
    pd
    .DataFrame(final_results_lm)
    .groupby('name')
    .agg(['mean', 'std'])
    .transpose()
)

name                                    OLS        RR
fit_time                     mean  0.022639  5.406434
                             std   0.002337  0.496836
score_time                   mean  0.004975  0.004921
                             std   0.002187  0.001235
test_r2                      mean  0.520441  0.536488
                             std   0.028013  0.021415
test_root_mean_squared_error mean  2.230271  2.223921
                             std   0.089971  0.087914

#### 2.4.2 Modelos Não-Lineares

In [52]:
(
    pd
    .DataFrame(final_results_nlm)
    .groupby('name')
    .agg(['mean', 'std'])
    .transpose()
)

name                                    KNN       SVR
fit_time                     mean  6.134819  8.197040
                             std   1.252128  0.699592
score_time                   mean  0.026955  0.280972
                             std   0.005692  0.037431
test_r2                      mean  0.521831  0.524043
                             std   0.017963  0.016654
test_root_mean_squared_error mean  2.262508  2.226751
                             std   0.108292  0.066687

### O melhor modelo obtido foi : Ridge Regression

### 2.6 Exportando o modelo

In [17]:
# Salva o modelo em disco
#joblib.dump(approach, '../models/model.joblib')  # executar somente após revisão dos dados